# Taller 5: Apache Spark con Databricks

## Definición de la entrega

Siga la siguiente guía. Al finalizar, deberá realizar un informe de formato libre donde se evidencie la ejecución de los pasos definidos en esta guía (Captura de pantalla), para las partes de *SparkSQL*: Manipulación de datos con *DataFrame*, Manipulación de datos con *RDD* y la exportación de resultados.

## Registro

Ingrese a [Databricks](https://www.databricks.com/try-databricks) y regístrese, en caso de obtener error al registrarse con un correo institucional, vuelva a intentar con un correo personal:

<img src="https://drive.google.com/uc?export=view&id=1qAtqFvX5vYjINV3Kpslc-9OXYzco8-7K" width="80%">

Seleccione *Get started with Community Edition*:

<img src="https://drive.google.com/uc?export=view&id=10YQ-JPtXqX7oLee6_0Uh34_9hP-Qap2x" width="80%">

Recibirá un email enviado por *Databricks* para confirmar su dirección de correo. De click al link correspondiente recibido en el email para la *confirmación* y asigne una contraseña a su cuenta. Después de estos dos pasos, será redireccionado automáticamente a una vista como la siguiente: 

<img src="https://drive.google.com/uc?export=view&id=17iegUuvHoSRHUhXTuJPbvV7BRgkD0Wno" width="80%">

Cree un nuevo *Cluster* mediante **Create -> Cluster**:

<img src="https://drive.google.com/uc?export=view&id=1gkpphdWaEOAkFJ-bb3fuCwpH6_ud_xTA" width="80%">

Asígnele el nombre de **MLDS-test**:

<img src="https://drive.google.com/uc?export=view&id=1OpsXRtAw-RNFGn-fmmrMDlIL7neGz4lF" width="80%">

Su vista en *Databricks* será:

<img src="https://drive.google.com/uc?export=view&id=1iuLGZmtlueXACaPXev040cBIPPnvuDCf" width="80%">

Ahora cree un Notebook mediante **Create -> Notebook**:

<img src="https://drive.google.com/uc?export=view&id=1d3cGOl90wCU7xOOoehdLqOEoO3jwmJTn" width="80%">

Asígnele el nombre de **Taller 5: Spark**:

<img src="https://drive.google.com/uc?export=view&id=1ky2qQ7lNiUUp6IkwnEkIO2NtzvPugqRF" width="80%">

Usted obtendrá una vista parecida a los notebooks de *Google Colaboratory*, por ejemplo, a continuación se puede notar una celda donde se ejecuta la función `print`:

<img src="https://drive.google.com/uc?export=view&id=1SyJqVAKT2D-3_YS2SCg9Zbv5HeDbQ3SO" width="80%">

## Spark SQL: Manipulación de datos con DataFrame

Este entorno cuenta con un archivo CSV de ejemplo, que podemos leer como un **DataFrame** de *Spark*:

```python
diamonds = sqlContext.read.format('com.databricks.spark.csv') \
             .options(header = 'true', inferSchema='true') \
             .load('/databricks-datasets/Rdatasets/data-001/csv/ggplot2/diamonds.csv')
```

Al ser escrito en el lenguaje de programación *Python*, puede realizar operaciones como la consulta del tipo de dato. Note que este objeto no es el mismo que un *DataFrame* de la librería *Pandas*:

```python
type(diamonds)

Out[2]: pyspark.sql.dataframe.DataFrame
```

Agregue una nueva celda y visualice el contenido del archivo CSV:

```python
display(diamonds)
```

<img src="https://drive.google.com/uc?export=view&id=1Db6TeDpAL1A2-sG08LT_jzSg57CaSVKH" width="100%">

Podemos mostrar el esquema del **DataFrame**. Esto equivale a realizar una descripción de la tabla, con los nombres de las variables y sus tipos de datos.

```python
diamonds.printSchema()
```

<img src="https://drive.google.com/uc?export=view&id=1UHytK9cBXmPpowwaHAWS2T_uIs_Ge6mk" width="30%">

Puede contar el número de filas en el conjunto de datos con el método **`count`**:

```python
print(diamonds.count())
```

Puede obtener los valores únicos de una columna con el método **`distinct`**. En este caso para ver los distintos colores de diamantes en el conjunto de datos:

```python
display(diamonds.select('color').distinct().collect())
```

Puede crear un nuevo **DataFrame** con la columna **`price`** de tipo **Double**. En este caso se realiza un cambio de tipo para una de las columnas y se guarda dentro de otro **DataFrame**:

```python
from pyspark.sql.types import DoubleType

diamondsCast = diamonds.withColumn("price", diamonds["price"].cast(DoubleType()))
```

Podemos realizar operaciones de agrupación y agregación, como el cálculo del precio promedio por valor **`carat`**, cambiando el nombre de la columna del resultado y ordenando por precio:

```python
from pyspark.sql.functions import *
carat_avgPrice = (diamondsCast
                .groupBy("carat")
                .avg("price")
                .withColumnRenamed("avg(price)", "avgPrice")
                .orderBy(desc("avgPrice")))
```

Finalmente, podemos consultar el DataFrame obtenido y ver el top-10 de los precios promedio más altos para el valor **`carat`**.

```python
carat_avgPrice.show(10)
```

<img src="https://drive.google.com/uc?export=view&id=1cpko1n3P-_numARRRRk9BVeJ421I9fi_" width="20%">

Para realizar visualizaciones rápidas sobre los datos, puede utilizar las opciones proporcionadas por el notebook, para ello utilice el botón que está en la parte superior de la tabla que es igual al símbolo aritmético de suma **+** después de ejecutar la siguiente sentencia:

```python
display(diamonds)
```

A continuación el símbolo:

<img src="https://drive.google.com/uc?export=view&id=1DxU3WLz_J1jztH1PXIQjQ3sunkILWHYf" width="80%">

Luego de dar click a una de las visualizaciones, por ejemplo, en *Legacy Visualización*:

<img src="https://drive.google.com/uc?export=view&id=1C-x9gr9QaioWBh4SUH9zmMvcXyT1oUHL" width="50%">

Luego de aplicar las configuraciones deseadas se obtiene:

<img src="https://drive.google.com/uc?export=view&id=1_E2kUVNUX8v0TLzFsEaP-KIKA4iV1ete" width="80%">

## Manipulación de datos con RDD

1. Se puede obtener un **RDD** directamente del **DataFrame**. 

```python
diamonds_rdd = diamonds.rdd
```

2. Ahora, podemos realizar operaciones sobre este objeto, como las siguientes:

> 2.1 Obtener los primeros tres elementos del **RDD**:

```python
diamonds_rdd.take(3)
```

> 2.2 Contar diamantes por tipo de corte **`cut`** con las funciones **`map`** y **`reduceByKey`**:

```python
countByGroup = diamonds_rdd.map(lambda x: (x.cut, 1)).reduceByKey(lambda x,y:x+y)
display(countByGroup.collect())
```

> 2.3 Seleccionar los distintos tipos de **`clarity`** para los diamantes en el conjunto de datos:

```python
distinctClarity = diamonds_rdd.map(lambda x: x.clarity).distinct()
distinctClarity.collect()
```

> 2.4 Calcular el precio promedio de los diamantes por **`cut`**. Note que esta operación es equivalente a una agregación usando funciones `Map`, `Reduce` y `Finalize` (con la función **`mapValues`**).

```python
avgPrice = diamonds_rdd.map(lambda x: (x.cut, 
                                    {'price': float(x.price), 'count': 1})). \
                reduceByKey(lambda x, y: {'price': x['price'] + y['price'], 
                                          'count': x['count'] + y['count']}). \
                mapValues(lambda x: x['price']/x['count'])

display(avgPrice.collect())
```

## Exportar los resultados a CSV

Tras realizar las operaciones y modificaciones correspondientes, podemos generar y descargar un archivo con el contenido. Para esto, seguiremos los siguientes pasos:

1. Eliminar el archivo si ya existe, pues se produce un error si un archivo con el mismo nombre ya existe:

```python
%fs rm -r /FileStore/carat_avgPrice
```

2. Guardar la consulta al archivo CSV en la ruta **FileStore**. Cada partición será guardada en un archivo individual. Usando **`repartition(1)`** se creará un único archivo de salida:

```python
(carat_avgPrice.repartition(1)
 .write
 .format('com.databricks.spark.csv')
 .options(header='true')
 .save('/FileStore/carat_avgPrice'))
```

3. También podemos seleccionar unas columnas específicas y almacenar solo los resultados relevantes. En este caso, retornaremos los campos **`cut`** y **`price`** del dataset original:

```python
%fs rm -r /FileStore/cutPrice
```

```python
(diamonds.repartition(1)
.select('cut', 'price')
.write
.format('com.databricks.spark.csv')
.options(header='true'
).save('/FileStore/cutPrice'))
```

4. Finalmente, podemos acceder a los archivos en las siguientes rutas:
> * `/FileStore/carat_avgPrice`
> * `/FileStore/cutPrice`

5. Desde PySpark podemos también leer y mostrar el CSV guardado anteriormente. Los puede descargar dando clic en el ícono:

<img src="https://drive.google.com/uc?export=view&id=10gEVnH1cEoC0HWxk2bs_eouqjG3Y2dyW" width="20%">

```python
cutPrice = sqlContext.read.format('com.databricks.spark.csv').options(header='true',
                                inferSchema='true').load('/FileStore/cutPrice')
display(cutPrice)

```

<img src="https://drive.google.com/uc?export=view&id=10KxLr3D9yvSLuRcRInWcO0lXzDiHa4r2" width="100%">

6. Puede también usar sintaxis *SQL* para consultar las tablas creadas:

```python
diamondsCast.registerTempTable("diamonds")
color_agg = sqlContext.sql("""
SELECT color, avg(price) AS price
FROM diamonds
GROUP BY color
ORDER BY color
""")
display(color_agg)
```

    Además, podemos realizar una consulta con una celda especial (creada con la etiqueta %sql), consultando directamente la tabla creada:

```python
%sql
SELECT color, avg(price) AS price FROM diamonds GROUP BY color ORDER BY color
```

## **Créditos**
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*